In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pysd
import math
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import pickle
import random
from sympy import symbols, limit, exp, factorial, oo
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.api import OLS
import statsmodels.api as sm

Data handling and preperation

In [2]:
#load csv data
file_path = '../../Data/Synthetic/Test5_sim100.csv'
df = pd.read_csv(file_path, index_col=0)
df


,D1,D2,K,O,T0 Effect,T1 Effect,T2 Effect,T3 Effect,T4 Effect,T5 Effect,...,T91 Reality,T92 Reality,T93 Reality,T94 Reality,T95 Reality,T96 Reality,T97 Reality,T98 Reality,T99 Reality,T100 Reality
Simulation,,,,,,,,,,,,,,,,,,,,,
1,19.541318,79.692584,-2.273493,1.054768,0.3574,-0.188882,-0.259431,-0.433748,-0.205381,-0.144525,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
2,18.519853,78.225166,3.740854,1.722982,0.3574,-0.188882,-0.252800,-0.405894,-0.136444,-0.008627,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
3,34.314858,34.208769,-3.036660,2.676776,0.3574,-0.188882,-0.252800,-0.408380,-0.148149,-0.040452,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
4,18.466592,78.802348,-1.972099,3.240107,0.3574,-0.188882,-0.252800,-0.408380,-0.148515,-0.042400,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
5,24.707870,30.612480,-0.546455,3.413189,0.3574,-0.188882,-0.252800,-0.408380,-0.148089,-0.040110,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,21.598787,5.573183,-0.163758,4.097775,0.3574,-0.188882,-0.252800,-0.408380,-0.148033,-0.039813,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
97,2.247931,71.415346,-3.685969,2.416931,0.3574,-0.188882,-0.252800,-0.574671,-0.663118,-0.985980,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
98,9.206257,47.061376,1.536814,2.714455,0.3574,-0.188882,-0.252800,-0.408380,-0.145003,-0.024954,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413


In [3]:
# keep the 4 first columns in a new dataframe called answers

answers = df.iloc[:, :4]
answers


,D1,D2,K,O
Simulation,,,,
1,19.541318,79.692584,-2.273493,1.054768
2,18.519853,78.225166,3.740854,1.722982
3,34.314858,34.208769,-3.036660,2.676776
4,18.466592,78.802348,-1.972099,3.240107
5,24.707870,30.612480,-0.546455,3.413189
...,...,...,...,...
96,21.598787,5.573183,-0.163758,4.097775
97,2.247931,71.415346,-3.685969,2.416931
98,9.206257,47.061376,1.536814,2.714455


In [4]:
#create a new dataframe called data that contains the rest of the columns
dat = df.iloc[:, 4:]
dat

,T0 Effect,T1 Effect,T2 Effect,T3 Effect,T4 Effect,T5 Effect,T6 Effect,T7 Effect,T8 Effect,T9 Effect,...,T91 Reality,T92 Reality,T93 Reality,T94 Reality,T95 Reality,T96 Reality,T97 Reality,T98 Reality,T99 Reality,T100 Reality
Simulation,,,,,,,,,,,,,,,,,,,,,
1,0.3574,-0.188882,-0.259431,-0.433748,-0.205381,-0.144525,-0.039208,-0.265472,-0.658320,-0.398658,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
2,0.3574,-0.188882,-0.252800,-0.405894,-0.136444,-0.008627,0.189119,0.072966,-0.178253,0.244226,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
3,0.3574,-0.188882,-0.252800,-0.408380,-0.148149,-0.040452,0.121788,-0.047552,-0.368118,-0.035823,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
4,0.3574,-0.188882,-0.252800,-0.408380,-0.148515,-0.042400,0.115927,-0.060944,-0.393671,-0.078390,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
5,0.3574,-0.188882,-0.252800,-0.408380,-0.148089,-0.040110,0.122896,-0.044825,-0.362506,-0.025686,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.3574,-0.188882,-0.252800,-0.408380,-0.148033,-0.039813,0.123790,-0.042775,-0.358565,-0.019052,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
97,0.3574,-0.188882,-0.252800,-0.574671,-0.663118,-0.985980,-1.340284,-1.951555,-2.471820,-2.688534,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413
98,0.3574,-0.188882,-0.252800,-0.408380,-0.145003,-0.024954,0.164966,0.044275,-0.204771,0.217442,...,0.233212,0.172299,0.099792,0.020667,-0.060975,-0.128101,-0.268079,-0.239575,-0.167486,-0.265413


In [5]:

# Assuming 'dat' is your original DataFrame with 100 simulations
# Each simulation run has 100 input values followed by 100 output values in each row

# Number of values per simulation (assuming 100 inputs and 100 outputs)
num_values = 101

# Creating a dictionary to hold each simulation run's DataFrame
simulation_dict = {}

for i in range(len(dat)):
    # Extracting input and output values for the current simulation
    outputs = dat.iloc[i, :num_values].values
    inputs = dat.iloc[i, num_values:].values
    
    # Creating a new DataFrame for the current simulation with two columns: 'input' and 'output'
    sim_df = pd.DataFrame({
        'A': inputs,
        'B': outputs
    })
    
    # Adding the new DataFrame to the dictionary
    simulation_dict[i+1] = sim_df
# Now, 'simulation_dict' contains all your simulations,
# with each key representing a simulation and its value being a DataFrame of inputs and outputs

    

No delay estimation

In [6]:
num = 100
for i in range(num):
    df = simulation_dict[i+1]
    results = []

    X = df['A'].values.reshape(-1, 1)
    y = df['B'].values

# Fitting the model using statsmodels
    model = sm.OLS(y, X).fit()

    # Getting the model's predictions
    y_pred = model.predict(X)

    # Calculating the Mean Squared Error manually, since statsmodels doesn't provide it directly
    mse = np.mean((y - y_pred) ** 2)

    # Extracting the model's coefficients
    # Note: In a model without an intercept, there's just one coefficient for the predictor
    coefficients = model.params[0]
    # Mean Absolute Error
    mae = mean_absolute_error(y, y_pred)
    # R-squared
    r_squared = model.rsquared
    # AIC
    aic = model.aic
    # BIC
    bic = model.bic
    # Confidence Intervals for the coefficients
    ci = model.conf_int()


    results.append({'mse': mse, 'k': coefficients, 'mae': mae, 'r_squared': r_squared, 'aic': aic, 'bic': bic, 'ci': ci})

    df_results = pd.DataFrame(results)
    df_sorted = df_results.sort_values(by='mse', ascending=True)
    top_estimation = df_sorted.iloc[0]

    answers.loc[i+1, 'best_mse'] = top_estimation['mse']
    answers.loc[i+1, 'best_k'] = top_estimation['k']
    answers.loc[i+1, 'best_mae'] = top_estimation['mae']
    answers.loc[i+1, 'best_r_squared'] = top_estimation['r_squared']
    answers.loc[i+1, 'best_aic'] = top_estimation['aic']
    answers.loc[i+1, 'best_bic'] = top_estimation['bic']
    #answers.loc[i+1, 'best_ci'] = top_estimation['ci']
    answers.loc[i+1, 'k_lower_limit'] = top_estimation['ci'][0][0]  # Assuming 'ci' is indexed [lower_bound, upper_bound]
    answers.loc[i+1, 'k_upper_limit'] = top_estimation['ci'][0][1]

    # Calculate and update 'answers' with the percentage errors
    #answers.loc[i+1, 'error_k'] = ((top_estimation['k'] - answers.loc[i+1, 'K']) / answers.loc[i+1, 'K']) 

    # Calculate and update 'answers' with the abs errors
    answers.loc[i+1, 'abs_error_k'] = abs(top_estimation['k'] - answers.loc[i+1, 'K'])



In [8]:
answers
#store the answers in a csv file
answers.to_csv('../../Data/Synthetic/no_delay_answers.csv')

Fixed delay estimation

In [7]:
def Fixed_delay6(values, i, d):
 
    df = values.copy()
    # Create a list to hold the lagged Series
    lagged_series_list = []

    # Create a new feature for each of the last i observations
    for k in range(1, i+1):
        lagged_series = values['A'].shift(k)
        lagged_series.name = f'A lag{k}'  
        lagged_series_list.append(lagged_series)

    # Concatenate all the Series along axis=1 to create a DataFrame
    lagged_values = pd.concat(lagged_series_list, axis=1)

    # Replace the NaN values with 0 in the new features
    lagged_values = lagged_values.fillna(0)
    
    # Create a weight vector of zeros with the d-th element set to 1
    weights = np.zeros(i)  # Create a zero vector of size i
    if 0 < d <= i:  # Ensure d is within the valid range
        weights[d-1] = 1  # Set the d-th position to 1 (adjust by -1 for 0-based indexing)

    weights = weights.reshape(1, -1)
    weights_transposed = weights.T

    # Compute the dot product of the two matrices
    result = np.dot(lagged_values.values, weights_transposed).reshape(-1) 

    X = result.reshape(-1, 1)
    y = df['B'].values

    # Fitting the model using statsmodels
    model = sm.OLS(y, X).fit()

    # Getting the model's predictions
    y_pred = model.predict(X)

    # Calculating the Mean Squared Error manually, since statsmodels doesn't provide it directly
    mse = np.mean((y - y_pred) ** 2)

    # Extracting the model's coefficients
    # Note: In a model without an intercept, there's just one coefficient for the predictor
    coefficients = model.params[0]
    # Mean Absolute Error
    mae = mean_absolute_error(y, y_pred)
    # R-squared
    r_squared = model.rsquared
    # AIC
    aic = model.aic
    # BIC
    bic = model.bic
    # Confidence Intervals for the coefficients
    ci = model.conf_int()

    return mse, coefficients, mae, r_squared, aic, bic, ci

In [8]:
num = 100
for i in range(num):
    values = simulation_dict[i+1]
    results = []

    # Iterate over the parameter space
    for delay1 in range(1, 91):
        # Call your modified function that also returns SSE
        (mse, k, mae, r2, aic, bic, ci) = Fixed_delay6(values, 100, delay1)

        results.append({ 'delay1': delay1, 'mse': mse, 'k': k, 'mae': mae, 'r2': r2, 'aic': aic, 'bic': bic, 'ci': ci})
        print('sim:',i, 'delay1:', delay1)


    df_results = pd.DataFrame(results)
    df_sorted = df_results.sort_values(by='mse', ascending=True)
    top_estimation = df_sorted.iloc[0]

    answers.loc[i+1, 'best_delay1'] = top_estimation['delay1']
    answers.loc[i+1, 'best_mse'] = top_estimation['mse']
    answers.loc[i+1, 'best_k'] = top_estimation['k']
    answers.loc[i+1, 'best_mae'] = top_estimation['mae']
    answers.loc[i+1, 'best_r2'] = top_estimation['r2']
    answers.loc[i+1, 'best_aic'] = top_estimation['aic']
    answers.loc[i+1, 'best_bic'] = top_estimation['bic']
    #answers.loc[i+1, 'best_ci'] = top_estimation['ci']
    answers.loc[i+1, 'k_lower_limit'] = top_estimation['ci'][0][0]  # Assuming 'ci' is indexed [lower_bound, upper_bound]
    answers.loc[i+1, 'k_upper_limit'] = top_estimation['ci'][0][1]

    # Calculate and update 'answers' with the abs errors
    answers.loc[i+1, 'abs_error_delay1'] = abs(top_estimation['delay1'] - answers.loc[i+1, 'D1'])
    answers.loc[i+1, 'abs_error_k'] = abs(top_estimation['k'] - answers.loc[i+1, 'K'])




sim: 0 delay1: 1
sim: 0 delay1: 2
sim: 0 delay1: 3
sim: 0 delay1: 4
sim: 0 delay1: 5
sim: 0 delay1: 6
sim: 0 delay1: 7
sim: 0 delay1: 8
sim: 0 delay1: 9
sim: 0 delay1: 10
sim: 0 delay1: 11
sim: 0 delay1: 12
sim: 0 delay1: 13
sim: 0 delay1: 14
sim: 0 delay1: 15
sim: 0 delay1: 16
sim: 0 delay1: 17
sim: 0 delay1: 18
sim: 0 delay1: 19
sim: 0 delay1: 20
sim: 0 delay1: 21
sim: 0 delay1: 22
sim: 0 delay1: 23
sim: 0 delay1: 24
sim: 0 delay1: 25
sim: 0 delay1: 26
sim: 0 delay1: 27
sim: 0 delay1: 28
sim: 0 delay1: 29
sim: 0 delay1: 30
sim: 0 delay1: 31
sim: 0 delay1: 32
sim: 0 delay1: 33
sim: 0 delay1: 34
sim: 0 delay1: 35
sim: 0 delay1: 36
sim: 0 delay1: 37
sim: 0 delay1: 38
sim: 0 delay1: 39
sim: 0 delay1: 40
sim: 0 delay1: 41
sim: 0 delay1: 42
sim: 0 delay1: 43
sim: 0 delay1: 44
sim: 0 delay1: 45
sim: 0 delay1: 46
sim: 0 delay1: 47
sim: 0 delay1: 48
sim: 0 delay1: 49
sim: 0 delay1: 50
sim: 0 delay1: 51
sim: 0 delay1: 52
sim: 0 delay1: 53
sim: 0 delay1: 54
sim: 0 delay1: 55
sim: 0 delay1: 56
s

In [ ]:
answers.to_csv('../../Data/Synthetic/fixed_answers.csv')

Symmetric delay estimation 

In [10]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS
import statsmodels.api as sm

def uni_delay6(values, d, o):
    df = values.copy()
    
    for i in range(0, o+1):
        column_name = f"B_est_{i}"
        df[column_name] = df["A"].astype(float)
    
    for i in range(0, len(values["A"])-1):
        for j in range(1, o + 1):
            prev_column = f'B_est_{j-1}'
            current_column = f'B_est_{j}'
            df.loc[i+1, current_column] = df.loc[i, current_column] + (df.loc[i, prev_column] - df.loc[i, current_column]) / (d/o)

    X = df[f"B_est_{o}"].values.reshape(-1, 1)
    y = df['B'].values

    # Fitting the model using statsmodels
    model = sm.OLS(y, X).fit()

    # Getting the model's predictions
    y_pred = model.predict(X)

    # Calculating the Mean Squared Error manually, since statsmodels doesn't provide it directly
    mse = np.mean((y - y_pred) ** 2)

    # Extracting the model's coefficients
    # Note: In a model without an intercept, there's just one coefficient for the predictor
    coefficients = model.params[0]
    # Mean Absolute Error
    mae = mean_absolute_error(y, y_pred)
    # R-squared
    r_squared = model.rsquared
    # AIC
    aic = model.aic
    # BIC
    bic = model.bic
    # Confidence Intervals for the coefficients
    ci = model.conf_int()

    return mse, coefficients, mae, r_squared, aic, bic, ci

In [ ]:
num = 100
for i in range(num):
    values = simulation_dict[i+1]
    results = []

    # Iterate over the parameter space
    for order in range(1, 6):
        for delay1 in range(1, 101):
            # Call your modified function that also returns SSE
            (mse, k, mae, r2, aic, bic, ci) = uni_delay6(values, delay1, order)

            print('sim:',i, 'order:', order, 'delay1:', delay1)
            results.append({'order': order, 'delay1': delay1, 'mse': mse, 'k': k, 'mae': mae, 'r2': r2, 'aic': aic, 'bic': bic, 'ci': ci})


    df_results = pd.DataFrame(results)
    df_sorted = df_results.sort_values(by='mse', ascending=True)
    top_estimation = df_sorted.iloc[0]

    answers.loc[i+1, 'best_order'] = top_estimation['order']
    answers.loc[i+1, 'best_delay1'] = top_estimation['delay1']
    answers.loc[i+1, 'best_mse'] = top_estimation['mse']
    answers.loc[i+1, 'best_k'] = top_estimation['k']
    answers.loc[i+1, 'best_mae'] = top_estimation['mae']
    answers.loc[i+1, 'best_r2'] = top_estimation['r2']
    answers.loc[i+1, 'best_aic'] = top_estimation['aic']
    answers.loc[i+1, 'best_bic'] = top_estimation['bic']
    #answers.loc[i+1, 'best_ci'] = top_estimation['ci']
    answers.loc[i+1, 'k_lower_limit'] = top_estimation['ci'][0][0]  # Assuming 'ci' is indexed [lower_bound, upper_bound]
    answers.loc[i+1, 'k_upper_limit'] = top_estimation['ci'][0][1]

    # Calculate and update 'answers' with the abs errors
    answers.loc[i+1, 'abs_error_order'] = abs(top_estimation['order'] - answers.loc[i+1, 'O'])
    answers.loc[i+1, 'abs_error_delay1'] = abs(top_estimation['delay1'] - answers.loc[i+1, 'D1'])
    answers.loc[i+1, 'abs_error_k'] = abs(top_estimation['k'] - answers.loc[i+1, 'K'])


In [ ]:
answers.to_csv('../../Data/Synthetic/Uni_answers.csv')

Asymmetric delay estimation

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def Asy_delay6(values, d1, d2, o):
    df = values.copy()
    o = round(o)  # Ensure 'o' is an integer

    # Creating shifted columns based on the order 'o'
    for i in range(0, o + 1):
        column_name = f"B_est_{i}"
        df[column_name] = df["A"].astype(float)

    # Applying the delay logic
    for i in range(0, len(df["A"]) - 1):
        if df.loc[i, 'A'] < df.loc[i, f'B_est_{o}']:
            d = d2
        else:
            d = d1

        for j in range(1, o + 1):
            prev_column = f'B_est_{j-1}'
            current_column = f'B_est_{j}'
            df.loc[i + 1, current_column] = df.loc[i, current_column] + (df.loc[i, prev_column] - df.loc[i, current_column]) / (d / o)

    # Preparing the X (independent variable) and y (dependent variable) arrays
    X = df[f"B_est_{o}"].values
    y = df['B'].values

    # Note: We're NOT adding a constant term to X, as we don't want an intercept in the model

    # Fitting the model using statsmodels
    model = sm.OLS(y, X).fit()

    # Getting the model's predictions
    y_pred = model.predict(X)

    # Calculating the Mean Squared Error manually, since statsmodels doesn't provide it directly
    mse = np.mean((y - y_pred) ** 2)

    # Extracting the model's coefficients
    # Note: In a model without an intercept, there's just one coefficient for the predictor
    coefficients = model.params[0]
    # Mean Absolute Error
    mae = mean_absolute_error(y, y_pred)
    # R-squared
    r_squared = model.rsquared
    # AIC
    aic = model.aic
    # BIC
    bic = model.bic
    # Confidence Intervals for the coefficients
    ci = model.conf_int()

    return mse, coefficients, mae, r_squared, aic, bic, ci


In [ ]:
def create_objective_function(values):
    def objective(params):
        delay1, delay2, order = map(int, params)  # Ensure parameters are integers
        mse, _, _, _, _, _, _= Asy_delay6(values, delay1, delay2, order)
        return mse
    return objective


In [ ]:
# Assuming 'simulation_dict' contains your datasets
# 'answers' DataFrame is initialized to store the results

# Define bounds for delay1, delay2, and order
benchmark_bounds = [(1, 51), (1, 101), (1, 6)]

# Adjusted loop to use differential_evolution
for i in range(100):  # Assuming you want to process simulations indexed from 1 to 10
    
    values = simulation_dict[i+1]  # Access the dataset for the current simulation
    
    # Create a customized objective function for the current dataset
    custom_objective = create_objective_function(values)
    
    # Perform the optimization using differential_evolution
    #result = differential_evolution(custom_objective, benchmark_bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7)
    result = differential_evolution(custom_objective, benchmark_bounds)
    
    # Extract the optimal parameter values
    optimal_delay1, optimal_delay2, optimal_order = result.x
    #optimal_delay1, optimal_delay2, optimal_order = int(optimal_delay1), int(optimal_delay2), int(optimal_order)  # Convert to integers if necessary
    
    # Recalculate mse and k using the optimal parameters
    mse, k, mae, r2, aic, bic, ci = Asy_delay6(values, optimal_delay1, optimal_delay2, optimal_order)
    
    # Update 'answers' DataFrame with the optimization results
    answers.loc[i+1, 'best_order'] = optimal_order
    answers.loc[i+1, 'best_delay1'] = optimal_delay1
    answers.loc[i+1, 'best_delay2'] = optimal_delay2
    answers.loc[i+1, 'best_mse'] = mse
    answers.loc[i+1, 'best_k'] = k
    answers.loc[i+1, 'best_mae'] = mae
    answers.loc[i+1, 'best_r2'] = r2
    answers.loc[i+1, 'best_aic'] = aic
    answers.loc[i+1, 'best_bic'] = bic
    answers.loc[i+1, 'k_lower_limit'] = ci[0][0]  # Assuming 'ci' is indexed [lower_bound, upper_bound]
    answers.loc[i+1, 'k_upper_limit'] = ci[0][1]
    
    #calculate and update 'answers' with the absolute errors
    answers.loc[i+1, 'abs_error_order'] = abs(optimal_order - answers.loc[i+1, 'O'])
    answers.loc[i+1, 'abs_error_delay1'] = abs(optimal_delay1 - answers.loc[i+1, 'D1'])
    answers.loc[i+1, 'abs_error_delay2'] = abs(optimal_delay2 - answers.loc[i+1, 'D2'])
    answers.loc[i+1, 'abs_error_k'] = abs(k - answers.loc[i+1, 'K'])

    print(f"Simulation {i+1} completed")


In [ ]:
answers.to_csv('../../Data/Synthetic/asy_answers.csv')